In [1]:
import os
import math

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import flwr as fl
import tensorflow as tf

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import Adadelta

from typing import Dict

from flwr.common.logger import log
from logging import INFO
from csv import writer


from numpy.random import seed
from tensorflow.keras.utils import set_random_seed

import matplotlib.pyplot as plt

In [2]:
def get_empresa_conductor(conductor_id):
    for i in range(1,4):
        if (conductor_id in CONDUCTORES_IDS[i]):
            return i

In [3]:
''' FUNCION PARA CARGAR LOS DATOS DE UN CLIENTE EN PARTICULAR '''
def prepare_model_data(client_file):
    df = pd.read_csv(client_file)
    
    train, test = train_test_split(df, test_size=0.30, random_state=42)
    
    X_train = train[['psd_delta', 'psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','eog_blinks', 'eog_var']]
    X_test = test[['psd_delta', 'psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','eog_blinks', 'eog_var']]
    y_train = train['y_class']
    y_test = test['y_class']
    
    scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [4]:
def cargar_dataset_varios_clientes(clientes):
    base_path = "./data/centralizado"
    
    X_train, X_val, y_train, y_val = prepare_model_data(f'{base_path}/cliente_{clientes[0]}.csv')
    
    for cid in clientes[1:]:
        path = f'{base_path}/cliente_{cid}.csv'
        X_train_act, X_val_act, y_train_act, y_val_act = prepare_model_data(path)
    
        X_train = np.vstack((X_train, X_train_act))
        X_val = np.vstack((X_val, X_val_act))
        y_train = np.concatenate((y_train, y_train_act))
        y_val = np.concatenate((y_val, y_val_act))
        
    return X_train, X_val, y_train, y_val

In [5]:
def get_model():
    # Model best hyperparameters (Ver notebook Hito0-Optimizacion-Baseline)
    neurons = 36
    activation = "relu"
    learning_rate = 0.180165
    optimizer = Adadelta(learning_rate=learning_rate)
    
    input_shape = (7,)
    
    # Create model
    model = Sequential()
    
    model.add(Dense(neurons, input_shape=input_shape, activation=activation))
    
    model.add(BatchNormalization())
        
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(neurons, activation=activation))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [6]:
def get_weights_from_file(path):
    a = np.load(path, allow_pickle=True)

    n_elems = [252, 36,
               36, 36, 36, 36,
               1296, 36,
               1296, 36,
               1296, 36,
               36, 1]

    weights = []

    # https://numpy.org/devdocs/reference/generated/numpy.lib.format.html#version-numbering
    # En base a la doc sabemos que los datos de interes estan al final, y como deben ser float32 => 4 bytes
    # por lo que se toman los n_elementos*4bytes del final

    for i, t in enumerate(a["arr_0"][0].tensors):
        act = np.frombuffer(t[-n_elems[i]*4:], dtype=np.float32)
        weights.append(act)

    # Se cambia la forma para que se adapte a la del modelo
    weights[0] = weights[0].reshape(7,36)
    weights[6] = weights[6].reshape(36,36)
    weights[8] = weights[8].reshape(36,36)
    weights[10] = weights[10].reshape(36,36)
    weights[12] = weights[12].reshape(36,1)
    
    return weights

# Level 1 functionality

In [7]:
class ConductorClient(fl.client.NumPyClient):
    def __init__(self, cid, empresa, model, x_train, y_train, x_val, y_val) -> None:
        self.cid = cid
        self.empresa = empresa
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        
        seed(1)
        set_random_seed(2)
        
        self.model.fit(self.x_train, self.y_train,
                       epochs=1,
                       batch_size=32,
                       verbose=0)
        
        return self.model.get_weights(), len(self.x_train), {"client": self.cid, "empresa": self.empresa}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=0)
        
        return loss, len(self.x_val), {"accuracy": acc, "client": self.cid, "empresa": self.empresa}

In [8]:
def conductor_fn(cid: str) -> fl.client.Client:
    model = get_model()
    
    empresa = get_empresa_conductor(int(cid))
    
    # Load data partition
    base_path = "./data/horizontal_v3/"
    path = f"{base_path}empresa_{empresa}/cliente_{cid}.csv"
    
    x_train_cid, x_val_cid, y_train_cid, y_val_cid = prepare_model_data(path)

    # Create and return client
    return ConductorClient(cid, empresa, model, x_train_cid, y_train_cid, x_val_cid, y_val_cid)

In [9]:
class L1SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(self, rnd, results, failures):
        aggregated_weights = super().aggregate_fit(rnd, results, failures)
        
        # COMPROBAR DE QUE EMPRESA ES EL CLIENTE
        emp_id = results[1].metrics["empresa"]
        
        if aggregated_weights is not None:
            # Save aggregated_weights
            print(f"Saving round {rnd} aggregated_weights...")
            np.savez(f"./rounds/hito3_L1-round-{rnd}-E{emp_id}-weights.npz", aggregated_weights)
            
        return aggregated_weights

    def aggregate_evaluate(self, rnd, results, failures):
        super_result = super().aggregate_evaluate(rnd, results, failures)
        
#         log(
#             INFO,
#             f"round-{rnd}-EVALUATION"
#         )
        
#         accuracy = []
#         data = {}
#         for r in results:
#             acc = r[1].metrics["accuracy"]
#             client = r[1].metrics["client"]
#             data[client] = acc
#             accuracy.append(acc)
        
#         df = pd.DataFrame(data, index=[0], columns=sorted(data.keys()))
#         df.to_csv(f"./results/hito1_v3-empresa_{EMPRESA_ACTUAL}.csv", mode='a', index=False, header=False)
        
#         log(
#             INFO,
#             sorted(data.items())
#         )
        
#         np.array(accuracy)
#         log(
#             INFO,
#             f"Aggregated accuracy: {np.mean(accuracy)} +- {np.std(accuracy)}"
#         )
        
        return super_result

# Level 2 functionality

In [10]:
class EmpresaClient(fl.client.NumPyClient):
    def __init__(self, cid, model, x_train, y_train, x_val, y_val) -> None:
        self.cid = cid
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self):
        return self.model.get_weights()

    def fit(self, parameters, config):
        emp_id = int(self.cid[-1])
        
        def L1_fit_config(rnd: int) -> Dict[str, str]:
            config = {
                "round": str(rnd)
            }
            return config
        
        fl.simulation.start_simulation(
        client_fn=conductor_fn,
        clients_ids=CONDUCTORES_IDS[emp_id],
        client_resources={"num_cpus": 1},
        num_rounds=1,
        strategy=L1SaveModelStrategy(
                min_available_clients = len(CONDUCTORES_IDS[emp_id]),
                min_fit_clients = len(CONDUCTORES_IDS[emp_id]),
                min_eval_clients = len(CONDUCTORES_IDS[emp_id]),
                on_fit_config_fn = L1_fit_config,
                on_evaluate_config_fn = L1_fit_config,
                accept_failures=False,
                initial_parameters=parameters
            )
        )
        
        # Se cargan los pesos agregados para la epoch de todos los conductores
        rnd = int(config["round"])
        path = f"./rounds/hito3_L1-round-{1}-E{emp_id}-weights.npz"
        
        weights = get_weights_from_file(path)
        
        # Se recuentan cuantas observaciones de train hay
        obs_total = len(self.x_train)
        
        return weights, obs_total, {"client": self.cid}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=0)
        
        # Se recuentan cuantas observaciones de test hay
        obs_total = len(self.x_val)
        
        return loss, obs_total, {"accuracy": acc, "client": self.cid}

In [11]:
def empresa_fn(cid: str) -> fl.client.Client:
    model = get_model()
    
    # Load data partition
    emp_id = int(cid[-1])
    x_train_cid, x_val_cid, y_train_cid, y_val_cid = cargar_dataset_varios_clientes(CONDUCTORES_IDS[emp_id])
    
    # Create and return client
    print(f"Creada empresa {cid}")
    return EmpresaClient(cid, model, x_train_cid, y_train_cid, x_val_cid, y_val_cid)

In [12]:
class L2SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(self, rnd, results, failures):
        aggregated_weights = super().aggregate_fit(rnd, results, failures)
        
        if aggregated_weights is not None:
            # Save aggregated_weights
            print(f"Saving round {rnd} aggregated_weights...")
            np.savez(f"./rounds/hito3_L2-round-{rnd}-weights.npz", aggregated_weights)
            
        return aggregated_weights

    def aggregate_evaluate(self, rnd, results, failures):
        super_result = super().aggregate_evaluate(rnd, results, failures)
        
        log(
            INFO,
            f"round-{rnd}-EVALUATION"
        )
        
        accuracy = []
        data = {}
        for r in results:
            acc = r[1].metrics["accuracy"]
            client = r[1].metrics["client"]
            data[client] = acc
            
            accuracy.append(acc)
        
        df = pd.DataFrame(data, index=[0], columns=sorted(data.keys()))
        df.to_csv(f"./results/hito3.csv", mode='a', index=False, header=False)
        
        log(
            INFO,
            sorted(data.items())
        )
        
        np.array(accuracy)
        log(
            INFO,
            f"Aggregated accuracy: {np.mean(accuracy)} +- {np.std(accuracy)}"
        )
        
        return super_result

In [13]:
seed(1)
set_random_seed(2)

model = get_model()

# Get model weights as a list of NumPy ndarray's
weights = model.get_weights()
# Serialize ndarrays to `Parameters`
parameters = fl.common.weights_to_parameters(weights)

In [14]:
# Ahora los clientes son las tres empresas (L2) y los conductores (L1)
global EMPRESAS_IDS
EMPRESAS_IDS = ["empresa_1", "empresa_2", "empresa_3"]

global CONDUCTORES_IDS
CONDUCTORES_IDS = {
    1: [3,4,5,12,  2,8,  1],
    2: [13,14,    6,7,10,11,16],
    3: [15,18,  9,17,20,21,  19]
}

 # Se inicializa el fichero de resultados
header = EMPRESAS_IDS

with open(f"./results/hito3.csv", 'w', ) as f:
    csv_writer = writer(f)
    csv_writer.writerow(header)

# Configuracion de parametros para el entrenamiento desde el servidor
def L2_fit_config(rnd: int) -> Dict[str, str]:
    config = {
        "round": str(rnd)
    }
    return config

# Start Flower simulation
    # min_fit_clients y min_eval_clients se ponen al maximo de clientes disponibles ya que estamos en un escenario controlado
fl.simulation.start_simulation(
    client_fn=empresa_fn,
    clients_ids=EMPRESAS_IDS,
    client_resources={"num_cpus": 3},
    num_rounds=1,
    strategy=L2SaveModelStrategy(
        min_available_clients = len(EMPRESAS_IDS),
        min_fit_clients = len(EMPRESAS_IDS),
        min_eval_clients = len(EMPRESAS_IDS),
        on_fit_config_fn = L2_fit_config,
        on_evaluate_config_fn = L2_fit_config,
        accept_failures=False,
        initial_parameters=parameters
    )
)

INFO flower 2022-05-02 18:28:29,766 | app.py:144 | Ray initialized with resources: {'object_store_memory': 2527183257.0, 'CPU': 8.0, 'memory': 5054366516.0, 'GPU': 1.0, 'node:127.0.0.1': 1.0}
INFO flower 2022-05-02 18:28:29,768 | app.py:153 | Starting Flower simulation running: {'num_rounds': 1}
INFO flower 2022-05-02 18:28:29,768 | server.py:128 | Initializing global parameters
INFO flower 2022-05-02 18:28:29,769 | server.py:323 | Using initial parameters provided by strategy
INFO flower 2022-05-02 18:28:29,769 | server.py:130 | Evaluating initial parameters
INFO flower 2022-05-02 18:28:29,769 | server.py:143 | FL starting
DEBUG flower 2022-05-02 18:28:29,770 | server.py:265 | fit_round: strategy sampled 3 clients (out of 3)


 pid=6264) Creada empresa empresa_2
 pid=15432) Creada empresa empresa_1


 pid=6264) [2022-05-02 18:28:35,324 C 6264 8948] core_worker_process.cc:29:  Check failed: !core_worker_process The process is already initialized for core worker.
 pid=6264) *** StackTrace Information ***
 pid=6264)     PyInit__raylet
 pid=6264)     PyInit__raylet
 pid=6264)     PyInit__raylet
 pid=6264)     PyType_GenericNew
 pid=6264)     PyEval_EvalFrameDefault
 pid=6264)     PyObject_GC_Del
 pid=6264)     PyFunction_Vectorcall
 pid=6264)     PyEval_EvalFrameDefault
 pid=6264)     PyObject_GC_Del
 pid=6264)     PyFunction_Vectorcall
 pid=6264)     PyVectorcall_Call
 pid=6264)     PyObject_Call
 pid=6264)     PyEval_EvalFrameDefault
 pid=6264)     PyObject_GC_Del
 pid=6264)     PyFunction_Vectorcall
 pid=6264)     PyVectorcall_Call
 pid=6264)     PyObject_Call
 pid=6264)     PyEval_EvalFrameDefault
 pid=6264)     PyObject_GC_Del
 pid=6264)     PyFunction_Vectorcall
 pid=6264)     PyEval_EvalFrameDefault
 pid=6264)     PyEval_EvalFrameDefault
 pid=6264)     PyEval_EvalFrameDefault
 p

 pid=19408) Creada empresa empresa_3
 pid=19720) Creada empresa empresa_2


 pid=19408) [2022-05-02 18:28:41,196 C 19408 19672] core_worker_process.cc:29:  Check failed: !core_worker_process The process is already initialized for core worker.
 pid=19408) *** StackTrace Information ***
 pid=19408)     PyInit__raylet
 pid=19408)     PyInit__raylet
 pid=19408)     PyInit__raylet
 pid=19408)     PyType_GenericNew
 pid=19408)     PyEval_EvalFrameDefault
 pid=19408)     PyObject_GC_Del
 pid=19408)     PyFunction_Vectorcall
 pid=19408)     PyEval_EvalFrameDefault
 pid=19408)     PyObject_GC_Del
 pid=19408)     PyFunction_Vectorcall
 pid=19408)     PyVectorcall_Call
 pid=19408)     PyObject_Call
 pid=19408)     PyEval_EvalFrameDefault
 pid=19408)     PyObject_GC_Del
 pid=19408)     PyFunction_Vectorcall
 pid=19408)     PyVectorcall_Call
 pid=19408)     PyObject_Call
 pid=19408)     PyEval_EvalFrameDefault
 pid=19408)     PyObject_GC_Del
 pid=19408)     PyFunction_Vectorcall
 pid=19408)     PyEval_EvalFrameDefault
 pid=19408)     PyEval_EvalFrameDefault
 pid=19408)    

...

 pid=18268) Creada empresa empresa_3


 pid=18268) [2022-05-02 18:29:04,523 C 18268 3228] core_worker_process.cc:29:  Check failed: !core_worker_process The process is already initialized for core worker.
 pid=18268) *** StackTrace Information ***
 pid=18268)     PyInit__raylet
 pid=18268)     PyInit__raylet
 pid=18268)     PyInit__raylet
 pid=18268)     PyType_GenericNew
 pid=18268)     PyEval_EvalFrameDefault
 pid=18268)     PyObject_GC_Del
 pid=18268)     PyFunction_Vectorcall
 pid=18268)     PyEval_EvalFrameDefault
 pid=18268)     PyObject_GC_Del
 pid=18268)     PyFunction_Vectorcall
 pid=18268)     PyVectorcall_Call
 pid=18268)     PyObject_Call
 pid=18268)     PyEval_EvalFrameDefault
 pid=18268)     PyObject_GC_Del
 pid=18268)     PyFunction_Vectorcall
 pid=18268)     PyVectorcall_Call
 pid=18268)     PyObject_Call
 pid=18268)     PyEval_EvalFrameDefault
 pid=18268)     PyObject_GC_Del
 pid=18268)     PyFunction_Vectorcall
 pid=18268)     PyEval_EvalFrameDefault
 pid=18268)     PyEval_EvalFrameDefault
 pid=18268)     

Saving round 1 aggregated_weights...
 pid=16260) Creada empresa empresa_3
 pid=15416) Creada empresa empresa_1
 pid=16260) Creada empresa empresa_2


DEBUG flower 2022-05-02 18:29:16,321 | server.py:223 | evaluate_round received 3 results and 0 failures
INFO flower 2022-05-02 18:29:16,322 | 3757911614.py:15 | round-1-EVALUATION
INFO flower 2022-05-02 18:29:16,325 | 3757911614.py:32 | [('empresa_1', 0.5615113973617554), ('empresa_2', 0.7681894898414612), ('empresa_3', 0.4327850341796875)]
INFO flower 2022-05-02 18:29:16,326 | 3757911614.py:38 | Aggregated accuracy: 0.5874953071276346 +- 0.138155491854851
INFO flower 2022-05-02 18:29:16,326 | server.py:182 | FL finished in 46.5561058
INFO flower 2022-05-02 18:29:16,327 | app.py:149 | app_fit: losses_distributed [(1, 0.6788757778575973)]
INFO flower 2022-05-02 18:29:16,327 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-05-02 18:29:16,328 | app.py:151 | app_fit: losses_centralized []
INFO flower 2022-05-02 18:29:16,328 | app.py:152 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.6788757778575973

In [15]:
#TODO, revisar todo y probar funcionamiento, ver Cpus, etc